In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader

from sklearn.metrics import confusion_matrix

from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Define the base model and the transformation to be applied to the data
model_base = 'googlenet'
transform = transforms.Compose([
    transforms.Resize((299,299) if model_base == 'inception' else (224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalization
])

# Load train and test datasets
train_dir = 'faces2/train/'
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dir = 'faces2/test/'
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

if model_base == 'resnet50':
  # Load ResNet50 pretrained model
  model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
elif model_base == 'inception':
  # Load Inception pretrained model
  model = models.inception_v3(weights=models.Inception_V3_Weights.DEFAULT)
elif model_base == 'googlenet':
  # Load Inception GoogLenet model
  model = models.googlenet(weights=models.GoogLeNet_Weights.DEFAULT)

# Define a label for the layers that will be updated
if model_base == 'resnet50':
    train_layer = 'layer4'
elif model_base == 'inception':
    train_layer = 'Mixed_7c'
elif model_base == 'googlenet':
    train_layer = 'inception5b'

for name, param in model.named_parameters():
    if train_layer in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

# Adapt the fully-connected layer (FC) to the number of output classes and add a new Softmax layer
num_classes = len(train_dataset.classes)
model.fc = nn.Sequential(nn.Linear(model.fc.in_features, num_classes), nn.Softmax(dim=1))

test_loader.dataset.class_to_idx

{'live': 0, 'spoof': 1}

In [3]:
# Move the model to GPU if possible
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

device

device(type='cuda', index=0)

In [4]:
# Define the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function to train the model
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in tqdm(train_loader, ncols=80):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            if model_base == 'inception':
                outputs, _ = model(inputs)
            else:
                outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Run the model training
train_model(model, train_loader, criterion, optimizer, num_epochs=10)

100%|████████████████████████████████████▉| 11762/11763 [18:51<00:00, 10.11it/s]/home/alex/DL/lib/python3.12/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
/home/alex/DL/lib/python3.12/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
100%|█████████████████████████████████████| 11763/11763 [18:51<00:00, 10.39it/s]


Epoch [1/10], Loss: 0.3781


100%|█████████████████████████████████████| 11763/11763 [18:31<00:00, 10.59it/s]


Epoch [2/10], Loss: 0.3648


100%|█████████████████████████████████████| 11763/11763 [18:01<00:00, 10.87it/s]


...

Epoch [8/10], Loss: 0.3448


100%|█████████████████████████████████████| 11763/11763 [18:14<00:00, 10.75it/s]


Epoch [9/10], Loss: 0.3429


100%|█████████████████████████████████████| 11763/11763 [18:15<00:00, 10.74it/s]

Epoch [10/10], Loss: 0.3416


In [5]:
# Function to test the model
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    groundtruth = []
    predictions = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            groundtruth.extend(list(labels.to('cpu').tolist()))
            predictions.extend(list(predicted.to('cpu').tolist()))
    
    print(f"Accuracy: {100 * correct / total:.2f}%")
    print('Confusion matrix:')
    print(confusion_matrix(groundtruth, predictions))

# Run the test
evaluate_model(model, test_loader)

Accuracy: 83.18%
Confusion matrix:
[[17693   914]
 [ 7684 24816]]


In [6]:
# Save the trained model
torch.save(model, model_base+'.pth')